In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import constants as cst
from datetime import timedelta, datetime, timezone
from matplotlib.colors import LogNorm, SymLogNorm, Normalize
from scipy.ndimage import gaussian_filter as gf


from scipy import stats

from multiprocess import Pool 



from spok.models import planetary as smp
from spok import smath as sm
from spok.coordinates import coordinates as scc
from spok import utils as su


from skimage.feature import hessian_matrix,hessian_matrix_eigvals
from scipy.interpolate import griddata

from sklearn.neighbors import KNeighborsRegressor

import speasy as spz
from speasy.products import SpeasyVariable
from typing import List

import spacepy.omni as spo

C:\Users\nithin\AppData\Local\Temp\ipykernel_19108\3484054293.py:31: UserWarning: Qin-Denton/OMNI2 data not found in current format. This module has limited functionality. Run spacepy.toolbox.update(QDomni=True) to download data.
  import spacepy.omni as spo


In [2]:
# Load OMNI data from SpacePy CDAWeb inventory
# And convert to DataFrame

cda=spz.inventories.tree.cda



In [5]:
data_test1= spz.get_data(
    [
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.BX_GSE,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.BY_GSM,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.BZ_GSM,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.proton_density,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.Vx,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.Vy,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.Vz,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.T,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.Pressure,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.Beta,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.Mach_num,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.flow_speed
    ]
    ,"2001-02-02 00:00","2021-05-26 11:00")
omni1 = data_test1[0].to_dataframe().join(data_test1[1].to_dataframe()).join(data_test1[2].to_dataframe()).join(data_test1[3].to_dataframe()) #B
omni1 = omni1.join(data_test1[4].to_dataframe()).join(data_test1[5].to_dataframe()).join(data_test1[6].to_dataframe()) #Vx,Vy,Vz GSE
omni1 = omni1.join(data_test1[7].to_dataframe()).join(data_test1[8].to_dataframe()).join(data_test1[9].to_dataframe())
omni1 = omni1.join(data_test1[10].to_dataframe())
omni1['B'] = np.sqrt(omni1['Bx, GSE']**2 + omni1['By, GSM']**2 + omni1['Bz, GSM']**2)
omni1 = omni1.join(data_test1[11].to_dataframe())
omni1

,"Bx, GSE","By, GSM","Bz, GSM",Proton density,"Vx Velocity, GSE","Vy Velocity, GSE","Vz Velocity, GSE",temperature,Flow pressure,Plasma Beta,Alfven Mach No.,B,"flow speed, GSE"
2001-02-02 00:00:00,-2.30,3.69,-2.69,3.82000,-436.299988,24.000000,-6.800000,75474.0,1.460000,1.24000,8.400000,5.112945,437.100006
2001-02-02 00:01:00,-2.73,3.59,-2.62,3.51000,-433.100006,24.799999,-10.000000,77424.0,1.320000,1.10000,7.800000,5.215879,433.899994
2001-02-02 00:02:00,-2.93,3.41,-2.32,3.58000,-432.100006,25.100000,-10.000000,77360.0,1.340000,1.19000,8.100000,5.059190,432.899994
2001-02-02 00:03:00,-2.48,3.88,-1.89,3.74000,-432.299988,25.000000,-11.000000,82377.0,1.400000,1.32000,8.400000,4.977640,433.200012
2001-02-02 00:04:00,-2.72,3.26,-2.37,3.79000,-432.600006,24.799999,-10.600000,82899.0,1.420000,1.38000,8.600000,4.862397,433.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-26 10:55:00,-0.44,1.82,-0.96,8.26000,-295.299988,-1.600000,-8.100000,11207.0,1.440000,9.23000,18.600000,2.104186,295.399994
2021-05-26 10:56:00,0.07,2.08,-1.12,999.98999,99999.898438,99999.898438,99999.898438,9999999.0,99.989998,999.98999,999.900024,2.363409,99999.898438
2021-05-26 10:57:00,-0.40,1.78,-0.88,7.76000,-295.399994,-2.200000,-7.500000,11174.0,1.360000,9.22000,18.600000,2.025537,295.500000
2021-05-26 10:58:00,0.25,2.16,-1.08,7.76000,-295.399994,-2.200000,-7.500000,11174.0,1.360000,7.63000,16.900000,2.427859,295.500000


In [6]:
data_test= spz.get_data(
    [
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.BX_GSE,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.BY_GSM,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.BZ_GSM,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.proton_density,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.Vx,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.Vy,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.Vz,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.T,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.Pressure,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.Beta,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.Mach_num,
        cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO2_1MIN.flow_speed
     ]
    ,"2001-02-02 00:00","2021-05-26 11:00")
omni2 = data_test[0].to_dataframe().join(data_test[1].to_dataframe()).join(data_test[2].to_dataframe()).join(data_test[3].to_dataframe()) #B
omni2 = omni2.join(data_test[4].to_dataframe()).join(data_test[5].to_dataframe()).join(data_test[6].to_dataframe()) #Vx,Vy,Vz GSE
omni2 = omni2.join(data_test[7].to_dataframe()).join(data_test[8].to_dataframe()).join(data_test[9].to_dataframe())
omni2 = omni2.join(data_test[10].to_dataframe())
omni2['B'] = np.sqrt(omni2['Bx, GSE']**2 + omni2['By, GSM']**2 + omni2['Bz, GSM']**2)
omni2 = omni2.join(data_test[11].to_dataframe())
omni2

,"Bx, GSE","By, GSM","Bz, GSM",Proton density,"Vx Velocity, GSE","Vy Velocity, GSE","Vz Velocity, GSE",temperature,Flow pressure,Plasma Beta,Alfven Mach No.,B,"flow speed, GSE"
2001-02-02 00:00:00,-2.30,3.69,-2.69,3.82000,-436.299988,24.000000,-6.800000,75474.0,1.460000,1.24000,8.400000,5.112945,437.100006
2001-02-02 00:01:00,-2.73,3.59,-2.62,3.51000,-433.100006,24.799999,-10.000000,77424.0,1.320000,1.10000,7.800000,5.215879,433.899994
2001-02-02 00:02:00,-2.93,3.41,-2.32,3.58000,-432.100006,25.100000,-10.000000,77360.0,1.340000,1.19000,8.100000,5.059190,432.899994
2001-02-02 00:03:00,-2.48,3.88,-1.89,3.74000,-432.299988,25.000000,-11.000000,82377.0,1.400000,1.32000,8.400000,4.977640,433.200012
2001-02-02 00:04:00,-2.72,3.26,-2.37,3.79000,-432.600006,24.799999,-10.600000,82899.0,1.420000,1.38000,8.600000,4.862397,433.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-26 10:55:00,-0.60,1.75,-0.93,8.55000,-294.799988,-1.900000,-4.600000,13126.0,1.490000,9.94000,19.200001,2.070604,294.899994
2021-05-26 10:56:00,0.20,2.15,-1.14,8.46000,-295.000000,-1.900000,-4.800000,12994.0,1.470000,8.29000,17.500000,2.441741,295.100006
2021-05-26 10:57:00,-0.44,1.75,-0.87,7.95000,-295.399994,-2.400000,-6.900000,12373.0,1.390000,9.79000,19.100000,2.003247,295.500000
2021-05-26 10:58:00,0.25,2.17,-1.09,7.95000,-295.399994,-2.400000,-6.900000,12372.0,1.390000,7.75000,17.000000,2.441209,295.500000


In [7]:
omni1 = omni1.rename(columns={
    'Bx, GSE': 'Bx',
    'By, GSM': 'By',
    'Bz, GSM': 'Bz',
    'Proton density': 'Np',
    'Vx Velocity, GSE': 'Vx',
    'Vy Velocity, GSE': 'Vy',
    'Vz Velocity, GSE': 'Vz',
    'temperature': 'Tp',
    'Flow pressure': 'Pd',
    'Plasma Beta': 'Beta',
    'Alfven Mach No.': 'Ma',
    'flow speed, GSE': 'V'
})

# Converting values beyond range to NaN
cols = ['Bx', 'By', 'Bz', 'Np','Beta','Ma','B']
for col in cols:
    omni1[col] = omni1[col].where(omni1[col] < 999, np.nan)

cols = ['Vx', 'Vy', 'Vz', 'V']
for col in cols:
    omni1[col] = omni1[col].where(omni1[col] < 99999, np.nan)

omni1['Tp'] = omni1['Tp'].where(omni1['Tp'] < 9999999, np.nan)
omni1['Pd'] = omni1['Pd'].where(omni1['Pd'] < 99, np.nan)

omni1_resampled = omni1.resample('5S').interpolate('nearest', limit=12, limit_direction='both') 

C:\Users\nithin\AppData\Local\Temp\ipykernel_19108\3986224963.py:28: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  omni1_resampled = omni1.resample('5S').interpolate('nearest', limit=12, limit_direction='both')


In [8]:
omni2 = omni2.rename(columns={
    'Bx, GSE': 'Bx',
    'By, GSM': 'By',
    'Bz, GSM': 'Bz',
    'Proton density': 'Np',
    'Vx Velocity, GSE': 'Vx',
    'Vy Velocity, GSE': 'Vy',
    'Vz Velocity, GSE': 'Vz',
    'temperature': 'Tp',
    'Flow pressure': 'Pd',
    'Plasma Beta': 'Beta',
    'Alfven Mach No.': 'Ma',
    'flow speed, GSE': 'V'
})

# Converting values beyond range to NaN
cols = ['Bx', 'By', 'Bz', 'Np','Beta','Ma','B']
for col in cols:
    omni2[col] = omni2[col].where(omni2[col] < 999, np.nan)

cols = ['Vx', 'Vy', 'Vz', 'V']
for col in cols:
    omni2[col] = omni2[col].where(omni2[col] < 99999, np.nan)

omni2['Tp'] = omni2['Tp'].where(omni2['Tp'] < 9999999, np.nan)
omni2['Pd'] = omni2['Pd'].where(omni2['Pd'] < 99, np.nan)

omni2_resampled = omni2.resample('5S').interpolate('nearest', limit=12, limit_direction='both') 

C:\Users\nithin\AppData\Local\Temp\ipykernel_19108\1509914695.py:28: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  omni2_resampled = omni2.resample('5S').interpolate('nearest', limit=12, limit_direction='both')


In [9]:
 # Save dataframes of omni1 and omni2 as pickle files
omni1_resampled.to_pickle('omni1_resampled.pkl')
omni2_resampled.to_pickle('omni2_resampled.pkl')

In [10]:
omni1_resampled

,Bx,By,Bz,Np,Vx,Vy,Vz,Tp,Pd,Beta,Ma,B,V
2001-02-02 00:00:00,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
2001-02-02 00:00:05,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
2001-02-02 00:00:10,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
2001-02-02 00:00:15,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
2001-02-02 00:00:20,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-26 10:58:40,0.28,2.17,-1.07,7.76,-295.399994,-2.2,-7.5,11174.0,1.36,7.57,16.9,2.435611,295.500000
2021-05-26 10:58:45,0.28,2.17,-1.07,7.76,-295.399994,-2.2,-7.5,11174.0,1.36,7.57,16.9,2.435611,295.500000
2021-05-26 10:58:50,0.28,2.17,-1.07,7.76,-295.399994,-2.2,-7.5,11174.0,1.36,7.57,16.9,2.435611,295.500000
2021-05-26 10:58:55,0.28,2.17,-1.07,7.76,-295.399994,-2.2,-7.5,11174.0,1.36,7.57,16.9,2.435611,295.500000


In [11]:
omni2_resampled


,Bx,By,Bz,Np,Vx,Vy,Vz,Tp,Pd,Beta,Ma,B,V
2001-02-02 00:00:00,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
2001-02-02 00:00:05,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
2001-02-02 00:00:10,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
2001-02-02 00:00:15,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
2001-02-02 00:00:20,-2.30,3.69,-2.69,3.82,-436.299988,24.0,-6.8,75474.0,1.46,1.24,8.4,5.112945,437.100006
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-26 10:58:40,0.24,2.15,-1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.409087,NaN
2021-05-26 10:58:45,0.24,2.15,-1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.409087,NaN
2021-05-26 10:58:50,0.24,2.15,-1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.409087,NaN
2021-05-26 10:58:55,0.24,2.15,-1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.409087,NaN
